## Heyyy Everyyoneeee !! *** pleasee UPVOTE if you liked it OR if you have a good heart *** 🥺 🥺 🥺

<img src="https://ih1.redbubble.net/image.1600796187.0360/mp,840x830,matte,f8f8f8,t-pad,1000x1000,f8f8f8.jpg" width="600" class="center"/>

In [ ]:
# ! pip install googletrans

In [ ]:
# ! pip install langdetect
# ! pip install translate
# ! pip install gensim
# ! python -m spacy download en_core_web_lg

In [ ]:
import spacy.cli
# spacy.cli.download("en_core_web_lg")
import en_core_web_lg
nlp = en_core_web_lg.load()

In [ ]:
## for data
import pandas as pd
import numpy as np

## for plotting
import matplotlib.pyplot as plt
import seaborn as sns

## for text processing
from dateutil import parser
import string
import re
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem.porter import PorterStemmer
from spacy.lang.en.stop_words import STOP_WORDS

from gensim.models import KeyedVectors

## for language detection
## warnings !!
import warnings
warnings.filterwarnings('ignore')


In [ ]:
import zipfile 

# zf = zipfile.ZipFile("/content/drive/My Drive/Data/ZS 2020.zip")

In [ ]:
train=pd.read_csv('../input/predicting-job-type-category-by-job-description/train.csv')
test=pd.read_csv('../input/predicting-job-type-category-by-job-description/test.csv')
submission=pd.read_csv('../input/predicting-job-type-category-by-job-description/sample_submission.csv')

In [ ]:
train.head() # we have 6 classes in 'job_type' and 11 classes in  'category'

In [ ]:
train.isnull().sum() # Verifying if needs any null modification !!

In [ ]:
train['job_description'][0]

In [ ]:
# as it can be seen the target class 'job_type' is highly imbalanced !!
#  'Any' and 'Part-Time' are below the means of consideration.

print('Class counts for Job_type : \n',train['job_type'].value_counts())

fig, ax = plt.subplots()
fig.suptitle('job_type', fontsize=12)
train['job_type'].reset_index().groupby('job_type').count().sort_values(by= 
       "index").plot(kind="barh", legend=False, 
        ax=ax).grid(axis='x')
plt.show()


In [ ]:
 # As it can be seen the target class 'category' is highly imbalanced !!
    
print('Class counts for category : \n',train['category'].value_counts())

fig, ax = plt.subplots()
fig.suptitle('category', fontsize=12)
train['category'].reset_index().groupby('category').count().sort_values(by= 
       "index").plot(kind="barh", legend=False, 
        ax=ax).grid(axis='x')
plt.show()

In [ ]:
# Lets focus on job description !!!

# Steps to be taken :
#
#     1. Text pre-processing:
#         1.1 Normalize case
#         1.2 Special character
#         1.3 Stop words
#         1.4 White space
#         1.5 Stemming
        
#     2. Feature extraction:
#         2.1 Word2vec embedding

In [ ]:
# Cleaning Process:

# For dates removal ~

def is_valid_date(date_str): # sub function
    try:
        parser.parse(date_str)
        return True
    except:
        return False
    
def date_removal(data):
    new_list = [' '.join([w for w in line.split() if not is_valid_date(w)]) for line in data]
    return (new_list[0])

def stemmer_and_stopWord(doc):
    
    doc= nlp(doc)
    token_list = []
    for token in doc:
      lemma = token.lemma_
      if lemma == '-PRON-' or lemma == 'be':
        lemma = token.text
      token_list.append(lemma)

    stemmed = token_list
    
    # Create list of word tokens after removing stopwords
    
    filtered_sentence =[] 
    for word in stemmed[:100]:
        lexeme = nlp.vocab[word]
        if lexeme.is_stop == False:
            filtered_sentence.append(word)
    return (' '.join(filtered_sentence))

def normaliz(filtered_sentence):
        
    words = [str(word).lower() for word in filtered_sentence.split()]
    return  ' '.join(words[:100])

# For number removal ~

def numbers_removal(data):
    s = [data]
    result = ''.join([i for i in s if not i.isdigit()])
    return (result)
# For punchuation & double white spaces ~

def punch_removal(words):
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in [words]]
    return re.sub(' +', ' ', stripped[:100][0])


def cleaner(data):
    string = [data]
    string = date_removal(string)
    # string = numbers_removal(string)
    string = punch_removal(string)
    string = stemmer_and_stopWord(string)
    string = normaliz(string)
    return string

In [ ]:
def cleaner(data):
    string = [data]
    string = date_removal(string)
    # string = numbers_removal(string)
    string = punch_removal(string)
    string = stemmer_and_stopWord(string)
    string = normaliz(string)
    return string

print('* --------------- for train data --------------- *')

list=[]
for i in train['job_description']: # cleansing !!
    list.append(cleaner(i))

train['job_description']=pd.Series(list) # updating the attributee !!

print('* --------------- for test data --------------- *')

list=[]
for i in test['job_description']: # cleansing !!
    list.append(cleaner(i))

test['job_description']=pd.Series(list) # updating the attributee !!


In [ ]:
## Feature extarction or EDA :

In [ ]:
# adding a new attribute for differencing the language of teh 'job_description' !!

import langdetect

print('* ------------------------- for train data-------------------------------- *')

train['lang'] = train["job_description"].apply(lambda x: langdetect.detect(x) if x.strip() != "" else "")

fig, ax = plt.subplots()
fig.suptitle('lang', fontsize=12)
train['lang'].reset_index().groupby('lang').count().sort_values(by= "index").plot(kind="barh", legend=False, ax=ax).grid(axis='x')
plt.show()

print('* -------------------------- for test data ------------------------------- *')

test['lang'] = test["job_description"].apply(lambda x: langdetect.detect(x) if x.strip() != "" else "")

fig, ax = plt.subplots()
fig.suptitle('lang', fontsize=12)
test['lang'].reset_index().groupby('lang').count().sort_values(by= "index").plot(kind="barh", legend=False, ax=ax).grid(axis='x')
plt.show()


In [ ]:
# from translate import Translator

from googletrans import Translator
translator = Translator()

In [ ]:
# Translating non english text into english : for train and test sets !!

print('* ------------------------- for train data-------------------------------- *')

all_eng = []
for index, lang in enumerate(train['lang']):
    if lang != 'en':
        es = translator.translate(train['job_description'][index])
        all_eng.append(es.text)
    else:
        all_eng.append(train['job_description'][index])

train['job_description']=pd.Series(all_eng)

print('* ------------------------- for test data-------------------------------- *')

all_eng_test = []
for index, lang in enumerate(test['lang']):
    if lang != 'en':
        es = translator.translate(test['job_description'][index])
        all_eng_test.append(es.text)
    else:
        all_eng_test.append(test['job_description'][index])

test['job_description']=pd.Series(all_eng_test)

In [ ]:
# Feature Extraction from the job discription :

print('* ------------------------- for train data-------------------------------- *')

# the relevance of these feature will be tested on the later stages !!

train['word_count'] = train["job_description"].apply(lambda x: 
                                                     len(str(x).split(" ")))
train['char_count'] = train["job_description"].apply(lambda x:
                                                     sum(len(word) 
                                                     for word in str(x).split(" ")))
train['sentence_count'] = train["job_description"].apply(lambda x: 
                                                         len(str(x).split(".")))
train['avg_word_length'] = train['char_count'] / train['word_count']
train['avg_sentence_lenght'] = train['word_count'] / train['sentence_count']

print('* ------------------------- for test data-------------------------------- *')

test['word_count'] = test["job_description"].apply(lambda x: 
                                                   len(str(x).split(" ")))
test['char_count'] = test["job_description"].apply(lambda x: 
                                                   sum(len(word) 
                                                   for word in str(x).split(" ")))
test['sentence_count'] = test["job_description"].apply(lambda x: 
                                                       len(str(x).split(".")))
test['avg_word_length'] = test['char_count'] / test['word_count']
test['avg_sentence_lenght'] = test['word_count'] / test['sentence_count']


In [ ]:
# A bivariate distributions to look at the correlation of a attribute with the target class !!

# conclusion : The categories have a similar length distribution but the samples have different sizes.

x = 'word_count'
y = 'job_type'

fig, ax = plt.subplots(nrows=1, ncols=2)
fig.suptitle(x, fontsize=12)
for i in train[y].unique():
    sns.distplot(train[train[y]==i][x], hist=True, kde=False, 
                 bins=10, hist_kws={"alpha":0.8}, 
                 axlabel="histogram", ax=ax[0])
    sns.distplot(train[train[y]==i][x], hist=False, kde=True, 
                 kde_kws={"shade":True}, axlabel="density",   
                 ax=ax[1])
ax[0].grid(True)
ax[0].legend(train[y].unique())
ax[1].grid(True)
plt.show()

In [ ]:
train.head()

In [ ]:
ig, ax = plt.subplots(1, 2, figsize=(18,4))

amount_val = train['word_count'].values
time_val = train['char_count'].values

sns.distplot(amount_val, ax=ax[0], color='red')
ax[0].set_title('Distribution of Transaction Amount', fontsize=14)

sns.distplot(time_val, ax=ax[1], color='blue')
ax[1].set_title('Distribution of Transaction Time', fontsize=14)

plt.show()

In [ ]:
x = 'word_count'
y = 'job_type'

sns.displot(train, x="word_count", hue="category")

In [ ]:
# adding a new attribute for differencing the language of teh 'job_description' !!

import langdetect

print('* ------------------------- for train data-------------------------------- *')

train['lang'] = train["job_description"].apply(lambda x: langdetect.detect(x) if x.strip() != "" else "")

fig, ax = plt.subplots()
fig.suptitle('lang', fontsize=12)
train['lang'].reset_index().groupby('lang').count().sort_values(by= "index").plot(kind="barh", legend=False, ax=ax).grid(axis='x')
plt.show()

print('* -------------------------- for test data ------------------------------- *')

test['lang'] = test["job_description"].apply(lambda x: langdetect.detect(x) if x.strip() != "" else "")

fig, ax = plt.subplots()
fig.suptitle('lang', fontsize=12)
test['lang'].reset_index().groupby('lang').count().sort_values(by= "index").plot(kind="barh", legend=False, ax=ax).grid(axis='x')
plt.show()


## Modeling :--->

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences 
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam



import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, MaxPooling1D, GlobalMaxPooling1D, LSTM, Dropout, GRU, Activation, Embedding, Bidirectional,SpatialDropout1D, BatchNormalization, Conv1D, MaxPool1D

In [ ]:
MAX_NB_WORDS = 28523 + 1
# Max number of words in each description.
MAX_SEQUENCE_LENGTH = 100
# This is fixed.

EMBEDDING_DIM = 100

Job_type = {
    'Permanent':0,
    'Contract/Interim':1,
    'Contract/Temp':2,
    'Temporary/Seasonal':3,
    'Any':4,
    'Part-Time':6
    }

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(train['job_description'].values)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

In [ ]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, 
                      filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', 
                      lower=True, oov_token='OOV')

tokenizer.fit_on_texts(train['job_description'].values)
X = tokenizer.texts_to_sequences(train['job_description'].values)

MAX_NB_WORDS = len(tokenizer.word_index) + 1 
# Max number of words in each description.
MAX_SEQUENCE_LENGTH = 100
# This is fixed.
EMBEDDING_DIM = 100

X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH) # Training labels

Y = pd.get_dummies(train.replace
                   ({"job_type": Job_type})
                   ['job_type'].values) # Target class

In [ ]:
X = tokenizer.texts_to_sequences(train['job_description'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

## 1. Training For Type !!!

In [ ]:
Job_type = {
    'Permanent':0,
    'Contract/Interim':1,
    'Contract/Temp':2,
    'Temporary/Seasonal':3,
    'Any':4,
    'Part-Time':6
    }


Y = pd.get_dummies(train.replace({"job_type": Job_type})['job_type'].values)
print('Shape of label tensor:', Y.shape)

In [ ]:
# Data Splitting
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, 
                                                    random_state = 42)

# Modeling
job_model = Sequential()
job_model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))

job_model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3, return_sequences=True))
job_model.add(LSTM(80, dropout=0.3, recurrent_dropout=0.3))

job_model.add(Dense(128, activation='relu'))
job_model.add(Dropout(0.3))

job_model.add(Dense(6, activation='softmax'))
job_model.compile(loss='categorical_crossentropy', optimizer='adam',
                  metrics=['accuracy'])

In [ ]:
class_w = {
    0:0.18616874,
    1:2.1041366,
    2:10.1969697,
    3:25.80825959,
    4:116.65333333,
    5:126.79710145
}

In [ ]:
job_model = Sequential()
job_model.add(Embedding(MAX_NB_WORDS, 300, input_length=X.shape[1]))
# job_model.add(SpatialDropout1D(0.2)) # flattening the 2D embedding into 1D

job_model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))

job_model.add(Dense(128, activation='relu'))
job_model.add(Dropout(0.3))

job_model.add(Dense(6, activation='softmax'))
job_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 10
batch_size = 64

history = job_model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,
                    callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)], class_weight = class_w)

In [ ]:
accr = job_model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
accr = job_model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show();

plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.legend()
plt.show();

## 2. Training For Category !!!

In [ ]:
Category = {
    'Pharmaceutical, Healthcare and Medical Sales':0,
    'Clinical Research':1,
    'Pharmaceutical Marketing':2,
    'Manufacturing & Operations':3,
    'Science':4,
    'Medical Affairs / Pharmaceutical Physician':5,
    'Regulatory Affairs':6,
    'Medical Information and Pharmacovigilance':7,
    'Data Management and Statistics':8,
    'Quality-assurance':9,
    'Pharmacy':10
    }
# Class weights :

class_w ={
    0:0.35310261, 
    1:0.50903273,
    2:1.486661  ,
    3:0.72404519,
    4:0.54833756,
    5:3.47320365,
    6:1.38324111,
    7:2.84566596,
    8:1.93283994,
    9:2.32562467,
    10:61.18181818
 }

Y = pd.get_dummies(train.replace({"category": Category})['category'].values)
print('Shape of label tensor:', Y.shape)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

In [ ]:
cat_model = Sequential()
cat_model.add(Embedding(MAX_NB_WORDS, 300, input_length=X.shape[1]))
# cat_model.add(SpatialDropout1D(0.2))

cat_model.add(LSTM(100, dropout=0.5, recurrent_dropout=0.5))

cat_model.add(Dense(128, activation='relu'))
cat_model.add(Dropout(0.5))

cat_model.add(Dense(11, activation='softmax'))
cat_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
epochs = 10
batch_size = 64

history = cat_model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,
                    callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

In [ ]:
accr = cat_model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show();

plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.legend()
plt.show();

## 3. Predicting the Type and Category : -->

In [ ]:
job_pred = []
category_pred = []

for i in test['job_description']:
    seq = tokenizer.texts_to_sequences([i]) # Using the same fitted tokenizer 
                                          # which was trained on the train data
    padded = pad_sequences(seq, maxlen = MAX_SEQUENCE_LENGTH)
    pred = job_model.predict(padded) # for job type
    labels = ['Permanent','Contract/Interim','Contract/Temp','Temporary/Seasonal',
            'Any','Part-Time']
    job_pred.append(labels[np.argmax(pred)])

    pred = cat_model.predict(padded) # for category
    labels = ['Pharmaceutical, Healthcare and Medical Sales','Clinical Research',
            'Pharmaceutical Marketing','Manufacturing & Operations','Science',
            'Medical Affairs / Pharmaceutical Physician','Regulatory Affairs',
            'Medical Information and Pharmacovigilance',
            'Data Management and Statistics','Quality-assurance','Pharmacy']
    category_pred.append(labels[np.argmax(pred)])

# creating submisison df
sub = pd.DataFrame()
sub['job_no'] = test['job_no']
sub['job_type']=pd.Series(job_pred)
sub['category']=pd.Series(category_pred)

In [ ]:
# creating submisison df

sub = pd.DataFrame()
sub['job_no'] = test ['job_no']

sub['job_type']=pd.Series(job_pred)
sub['category']=pd.Series(category_pred)

In [ ]:
sub.head()